In [7]:
pip install cryptography

In [8]:
import hashlib
import time
import json
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature
from typing import List, Optional

# 1. Wallet Class (to create public/private keys)
#     The Wallet class represents a user's digital wallet.
#     It is responsible for generating and managing cryptographic
#     key pairs (private key and public key) using Elliptic Curve Cryptography.
class Wallet:
    def __init__(self):
        # Generate a private key using the SECP256K1 elliptic curve
        self.private_key = ec.generate_private_key(ec.SECP256K1())
        # Derive the corresponding public key from the private key
        self.public_key = self.private_key.public_key()

    def get_address(self):
        # Convert the public key to a string (wallet address).
        pub_bytes = self.public_key.public_bytes(
            encoding=serialization.Encoding.X962,
            format=serialization.PublicFormat.UncompressedPoint
        )
        return pub_bytes.hex()

    def sign(self, data):
        # Sign data using the private key.
        return self.private_key.sign(data.encode(), ec.ECDSA(hashes.SHA256()))

    def get_private_key(self):
        return self.private_key



# ------------------------------------------------------------



# 2. Transaction Class

class Transaction:
    # The Transaction class represents a transfer of value between two addresses.
    # Each transaction must be cryptographically signed by the sender
    # to ensure integrity and authorization.
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.signature = None

    def calculate_hash(self):
        # Generate a SHA-256 hash of the transaction data
        data = f"{self.from_address}{self.to_address}{self.amount}"
        return hashlib.sha256(data.encode()).hexdigest()

    def sign_transaction(self, wallet):
        # Sign the transaction using the wallet's private key
        if self.from_address != wallet.get_address():
            raise Exception("You cannot sign transactions for other wallets!")
        tx_hash = self.calculate_hash()
        self.signature = wallet.sign(tx_hash)

    def is_valid(self):
        # Check if the transaction is valid (signed correctly
        if self.from_address is None:  # Mining reward transaction
            return True
        if not self.signature:
            raise Exception("No signature in this transaction!")

        tx_hash = self.calculate_hash()
        try:
            pub_key_bytes = bytes.fromhex(self.from_address)
            public_key = ec.EllipticCurvePublicKey.from_encoded_point(
                ec.SECP256K1(), pub_key_bytes
            )
            public_key.verify(self.signature, tx_hash.encode(), ec.ECDSA(hashes.SHA256()))
            return True
        except InvalidSignature:
            return False

    def to_dict(self):
        # Convert transaction data to dictionary format
        return {
            "from": self.from_address,
            "to": self.to_address,
            "amount": self.amount
        }


# ------------------------------------------------------------




# 3. Block Class

class Block:
    # The Block class represents a block in the blockchain.
    # Each block contains a list of transactions and is linked
    # to the previous block through a cryptographic hash.
    def __init__(self, timestamp, transactions):
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = ""
        self.hash = ""
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        # Calculate SHA-256 hash of the block
        data = str(self.timestamp) + str([tx.to_dict() for tx in self.transactions]) + self.previous_hash + str(self.nonce)
        return hashlib.sha256(data.encode()).hexdigest()

    def mine_block(self, difficulty):
        # Proof of Work: mine the block until the hash has required leading zeros
        target = "0" * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"Block mined: {self.hash}")

    def has_valid_transactions(self):
        # Check if all transactions in this block are valid
        for tx in self.transactions:
            if not tx.is_valid():
                return False
        return True



# ------------------------------------------------------------



# 4. Blockchain Class

class Blockchain:
    # The Blockchain class manages the entire blockchain system,
    # including block creation, mining, transaction handling, and chain validation.
    def __init__(self, difficulty: int = 2):
        self.chain = [self.create_genesis_block()]
        self.difficulty = difficulty
        self.pending_transactions = []
        self.mining_reward = 100.0

    def create_genesis_block(self):
        # Create the first block in the chain (Genesis Block
        return Block(time.time(), [])

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, transaction):
        # Add a new transaction to pending transactions
        if not transaction.from_address or not transaction.to_address:
            raise Exception("Transaction must include from and to address.")
        if not transaction.is_valid():
            raise Exception("Cannot add invalid transaction to chain.")
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self, mining_reward_address):
        # Mine a block containing pending transactions and reward the miner
        block = Block(time.time(), self.pending_transactions)
        block.previous_hash = self.get_latest_block().hash
        block.mine_block(self.difficulty)
        self.chain.append(block)
        # Reset pending transactions and add mining reward
        self.pending_transactions = [
            Transaction(None, mining_reward_address, self.mining_reward)
        ]

    def get_balance_of_address(self, address):
        # Calculate the balance of a given wallet address
        balance = 0.0
        for block in self.chain:
            for tx in block.transactions:
                if tx.from_address == address:
                    balance -= tx.amount
                if tx.to_address == address:
                    balance += tx.amount
        return balance

    def is_chain_valid(self):
        # Check if the blockchain is valid (no tampering)
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i-1]
            if not curr.has_valid_transactions():
                return False
            if curr.hash != curr.calculate_hash():
                return False
            if curr.previous_hash != prev.hash:
                return False
            if curr.hash[:self.difficulty] != "0" * self.difficulty:
                return False
        return True




# ------------------------------------------------------------



# 5. Testing

if __name__ == "__main__":
    # Create wallets
    wallet1 = Wallet()
    wallet2 = Wallet()
    addr1 = wallet1.get_address()
    addr2 = wallet2.get_address()

    print(" Wallet 1:", addr1[:30], "...")
    print(" Wallet 2:", addr2[:30], "...")

    # Create blockchain
    coin = Blockchain(difficulty=2)

    # Create and sign a transaction
    tx = Transaction(addr1, addr2, 10)
    tx.sign_transaction(wallet1)
    coin.add_transaction(tx)

    print("  Mining the first block...")
    coin.mine_pending_transactions(addr1)

    print(f" Balance of Wallet 1: {coin.get_balance_of_address(addr1)}")
    print(f" Is blockchain valid? {coin.is_chain_valid()}")
    print("  Mining the second block...")
    coin.mine_pending_transactions(addr1)
    print(f" Balance of Wallet 1: {coin.get_balance_of_address(addr1)}")

    # Attempt tampering
    coin.chain[1].transactions[0].amount = 999
    print(f" After tampering, is blockchain valid? {coin.is_chain_valid()}")



 Wallet 1: 044ae466ce026dbe713480d3987cc7 ...
 Wallet 2: 04d40be72c5387a8c4c9b3a785a781 ...
  Mining the first block...
Block mined: 00fb4dd6a98632a309d876bd835b34b8b7dc17bff955775d5486c0a74196986e
 Balance of Wallet 1: -10.0
 Is blockchain valid? True
  Mining the second block...
Block mined: 001a0528759f99ff4c8410118602336efdea3587d0e1589b63daa8213e9ccb5d
 Balance of Wallet 1: 90.0
 After tampering, is blockchain valid? False
